In [1]:
%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import sklearn
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.linear_model import Lasso

C:\Users\marco\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
tempo = pd.read_excel('meteorologia.xlsx')

tempo = tempo.groupby(['Estacao','Data'])
tempo = tempo.last()
tempo = tempo.dropna()
#tempo.head()


In [4]:
train = pd.DataFrame({'Hora': tempo['Hora'][:1500],
                      'Precipitação': tempo['Precipitacao'][:1500],
                      'Temperatura Mínima': tempo['TempMinima'][:1500],
                      'Temperatura Máxima': tempo['TempMaxima'][:1500],
                      'Insolação': tempo['Insolacao'][:1500],
                      'Umidade Relativa Média': tempo['Umidade Relativa Media'][:1500],
                      'Velocidade do Vento Média': tempo['Velocidade do Vento Media'][:1500]})

test = pd.DataFrame({ 'Hora': tempo['Hora'][1500:],
                      'Precipitação': tempo['Precipitacao'][1500:],
                      'Temperatura Mínima': tempo['TempMinima'][1500:],
                      'Temperatura Máxima': tempo['TempMaxima'][1500:],
                      'Insolação': tempo['Insolacao'][1500:],
                      'Umidade Relativa Média': tempo['Umidade Relativa Media'][1500:],
                      'Velocidade do Vento Média': tempo['Velocidade do Vento Media'][1500:]})

In [5]:
log = []


for i in range(len(tempo['Precipitacao'])):
    if tempo['Precipitacao'][i]==0:
        log.append(tempo['Precipitacao'][i])
    else:
        log.append(np.log(tempo['Precipitacao'][i]))



X_temp_media_dia_anterior = []
media_umidade = []
            

for i in range(1,len(tempo['TempMaxima'])):
    X_temp_media_dia_anterior.append((tempo['TempMinima'][i]+tempo['TempMaxima'][i])/2)
    media_umidade.append((tempo['Umidade Relativa Media'][i]+tempo['Umidade Relativa Media'][i-1])/2)


train = pd.DataFrame({'Hora': tempo['Hora'][2:1500],
                      'Precipitação': tempo['Precipitacao'][2:1500],
                      'Log_Precipitacao': log[2:1500],
                      'Temperatura Mínima': tempo['TempMinima'][2:1500],
                      'Temperatura Máxima': tempo['TempMaxima'][2:1500],
                      'Temperatura Média do Dia Anterior': X_temp_media_dia_anterior[0:1498],
                      'Insolação': tempo['Insolacao'][2:1500],
                      'Umidade Relativa Média': tempo['Umidade Relativa Media'][2:1500],
                      'Média de Umidade de Dois Dias Anteriores': media_umidade[0:1498],
                      'Velocidade do Vento Média': tempo['Velocidade do Vento Media'][2:1500]})

test = pd.DataFrame({ 'Hora': tempo['Hora'][1500:2745],
                      'Precipitação': tempo['Precipitacao'][1500:2745],
                      'Log_Precipitacao': log[1500:2745],
                      'Temperatura Mínima': tempo['TempMinima'][1500:2745],
                      'Temperatura Máxima': tempo['TempMaxima'][1500:2745],
                      'Temperatura Média do Dia Anterior': X_temp_media_dia_anterior[1498:2743],
                      'Insolação': tempo['Insolacao'][1500:2745],
                      'Umidade Relativa Média': tempo['Umidade Relativa Media'][1500:2745],
                      'Média de Umidade de Dois Dias Anteriores': media_umidade[1498:2743],
                      'Velocidade do Vento Média': tempo['Velocidade do Vento Media'][1500:2745]})
    

In [7]:
Y = train['Log_Precipitacao']


X = train[['Temperatura Média do Dia Anterior', 'Média de Umidade de Dois Dias Anteriores']]
X_teste = test[['Temperatura Média do Dia Anterior', 'Média de Umidade de Dois Dias Anteriores']]
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()

In [9]:
def Previsaodotempo(Tmax,Tmin,Umi):
    Tmed = (Tmax+Tmin)/2
    
    Umidadesp=[(52+96)/2,(100+57)/2,Umi]
    Tempsp=[27+20/2,(27+29)/2,Tmed]
    
    Y = train['Log_Precipitacao']


    X = train[['Temperatura Média do Dia Anterior', 'Média de Umidade de Dois Dias Anteriores']]
    X_teste = test[['Temperatura Média do Dia Anterior', 'Média de Umidade de Dois Dias Anteriores']]
    X = sm.add_constant(X)
    model = sm.OLS(Y,X)
    results = model.fit()

    xinho = pd.DataFrame({'Tmda':Tempsp,'MUdd':Umidadesp})
    #print(xinho)
    new_x = xinho[['Tmda','MUdd']]
    new_x = sm.add_constant(new_x)
    y_predict = results.predict(new_x)
    
    print(y_predict)
    
    if y_predict[2] > 0:
        return True
    else:
        return False

In [10]:
import tkinter as Tkinter
import random
 
class minhaApp_tk(Tkinter.Tk):

    def __init__(self,parent):
        Tkinter.Tk.__init__(self,parent)
        self.parent = parent
        self.initialize()
        
         
         
    def initialize(self):     
        self.grid()            
        
        self.instruLabel = Tkinter.Label(self,text="Insira os dados abaixo para descobrir se choverá amanhã", height = 2, font=("Helvetica", 10))
        self.instruLabel.grid(columnspan=3,column=0,row=0,sticky='EW') 

        
        self.lblC= Tkinter.Label(self, text="Temperatura Máxima")     
        self.lblC.grid(column=0, row=1)              
         
        self.lblSep= Tkinter.Label(self, text="Temperatura Mínima")     
        self.lblSep.grid(column=1, row=1)               
 
 
        self.lblF= Tkinter.Label(self, text="Média de Umidade entre ontem e hoje")     
        self.lblF.grid(column=2, row=1)               
 
        self.entrymax = Tkinter.Entry(self)           
        self.entrymax.grid(column=0,row=2,sticky='EW')
        
        self.entrymin = Tkinter.Entry(self)           
        self.entrymin.grid(column=1,row=2,sticky='EW')
 
        self.entryumi = Tkinter.Entry(self)           
        self.entryumi.grid(column=2,row=2,sticky='EW')
 
 
 
        self.btnCalculaF = Tkinter.Button(self,text=u"Prever o tempo",command=self.OnButtonCalculaChuva)  
                                                                   
        self.btnCalculaF.grid(column=1,row=3) 
        
        self.resuLabel = Tkinter.Label(self,text=" ", height = 2, font=("Helvetica", 28))
        self.resuLabel.grid(columnspan=3,column=0,row=5,sticky='EW') 

    def OnButtonCalculaChuva(self):     
        Tmax= float(self.entrymax.get())
        Tmin= float(self.entrymin.get()) 
        Umi= float(self.entryumi.get())  
        resultado = Previsaodotempo(Tmax,Tmin,Umi)
        if resultado == True:
            resultado = 'VAI CHOVER'
        else:
            resultado = 'NÃO VAI CHOVER'
        self.resuLabel.config(text=resultado)
 


if __name__ == "__main__":
    app = minhaApp_tk(None)     
    app.title('Choverá amanha?')
    app.mainloop()              

0    2.238786
1    1.526174
2   -3.288372
dtype: float64
